Import necesary libraries 

In [67]:
import os
import pandas as pd
import cv2
import random
from ultralytics import YOLO


Data Preperation 

only using the LARD_Train_Birk_LFST folder of data, since the data set is so large and will take to long to run. I will be dividing that data set into training, validation and test data

In [68]:
# Specify the training folder to use
root_folder = r'C:\Users\Annabelle\Desktop\runway-bounding-box-detection NEW'
train_folder = 'LARD_train_BIRK_LFST'

In [69]:
# Paths for images, metadata, and output
folder_path = os.path.join(root_folder, train_folder) #path to folder 'LARD_train_BIRK_LFST'
image_folder = os.path.join(folder_path, 'images') #path to images sub-folder in 'LARD_train_BIRK_LFST'
csv_path = os.path.join(folder_path, 'LARD_train_BIRK_LFST.csv')  # path to CSV file
output_folder = 'dataset_demo' #new folder for the sample of train & val images and labels taking for this
subset_size = 20  # Number of images to sample for train/val. doing small ammount to speed up demo/training time 

In [70]:
print("Images found in the folder:")
print(os.listdir(image_folder))  # List all images in the folder

# Confirm the number of images found
print(f"Total images found: {len(os.listdir(image_folder))}")


Images found in the folder:
['BIRK_01_500_000.jpeg', 'BIRK_01_500_001.jpeg', 'BIRK_01_500_002.jpeg', 'BIRK_01_500_003.jpeg', 'BIRK_01_500_004.jpeg', 'BIRK_01_500_005.jpeg', 'BIRK_01_500_006.jpeg', 'BIRK_01_500_007.jpeg', 'BIRK_01_500_008.jpeg', 'BIRK_01_500_009.jpeg', 'BIRK_01_500_010.jpeg', 'BIRK_01_500_011.jpeg', 'BIRK_01_500_012.jpeg', 'BIRK_01_500_013.jpeg', 'BIRK_01_500_014.jpeg', 'BIRK_01_500_015.jpeg', 'BIRK_01_500_016.jpeg', 'BIRK_01_500_017.jpeg', 'BIRK_01_500_018.jpeg', 'BIRK_01_500_019.jpeg', 'BIRK_01_500_020.jpeg', 'BIRK_01_500_021.jpeg', 'BIRK_01_500_022.jpeg', 'BIRK_01_500_023.jpeg', 'BIRK_01_500_024.jpeg', 'BIRK_01_500_025.jpeg', 'BIRK_01_500_026.jpeg', 'BIRK_01_500_027.jpeg', 'BIRK_01_500_028.jpeg', 'BIRK_01_500_029.jpeg', 'BIRK_01_500_030.jpeg', 'BIRK_01_500_031.jpeg', 'BIRK_01_500_032.jpeg', 'BIRK_01_500_033.jpeg', 'BIRK_01_500_034.jpeg', 'BIRK_01_500_035.jpeg', 'BIRK_01_500_036.jpeg', 'BIRK_01_500_037.jpeg', 'BIRK_01_500_038.jpeg', 'BIRK_01_500_039.jpeg', 'BIRK_01_50

In [71]:
# Create output directory structure
os.makedirs(f'{output_folder}/images/train', exist_ok=True)
os.makedirs(f'{output_folder}/images/val', exist_ok=True)
os.makedirs(f'{output_folder}/labels/train', exist_ok=True)
os.makedirs(f'{output_folder}/labels/val', exist_ok=True)

In [72]:
# Load the CSV metadata with semicolon separator
df = pd.read_csv(csv_path, sep=';')

In [73]:
# **New code to handle 'images/' prefix in CSV filenames**
df['image'] = df['image'].str.replace('images/', '', regex=False).str.strip()

# Verify the adjustment by printing the first few filenames
print("Adjusted Filenames from the CSV:")
print(df['image'].head())  # Confirm that 'images/' prefix is removed

# Collect all images from the folder and randomly sample a subset
images = [img for img in os.listdir(image_folder) if img.endswith(('.jpg', '.jpeg'))]
print(f"Total images found in folder: {len(images)}")

# Check for missing images
missing_files = [img for img in df['image'] if img not in images]
if missing_files:
    print(f"These images are listed in the CSV but not found in the folder: {missing_files}")
else:
    print("All CSV images match with folder images!")

Adjusted Filenames from the CSV:
0    BIRK_01_500_000.jpeg
1    BIRK_01_500_001.jpeg
2    BIRK_01_500_002.jpeg
3    BIRK_01_500_003.jpeg
4    BIRK_01_500_004.jpeg
Name: image, dtype: object
Total images found in folder: 1819
All CSV images match with folder images!


In [74]:
# Sample images for train and val sets
sampled_images = random.sample(images, min(subset_size, len(images)))
num_train = len(sampled_images) // 2
train_images = sampled_images[:num_train]
val_images = sampled_images[num_train:]


In [75]:
# Helper function to scale bounding boxes with image resizing
def scale_bbox(corners, original_size, new_size):
    orig_w, orig_h = original_size
    new_w, new_h = new_size
    x_scale = new_w / orig_w
    y_scale = new_h / orig_h

    # Scale each corner's X and Y coordinates
    return [
        corners[i] * x_scale if i % 2 == 0 else corners[i] * y_scale
        for i in range(8)
    ]

# Helper function to convert scaled corners to YOLO format
def convert_to_yolo(size, corners):
    img_w, img_h = size
    x_min = min([corners[i] for i in range(0, 8, 2)])
    x_max = max([corners[i] for i in range(0, 8, 2)])
    y_min = min([corners[i] for i in range(1, 8, 2)])
    y_max = max([corners[i] for i in range(1, 8, 2)])

    x_center = (x_min + x_max) / (2 * img_w)
    y_center = (y_min + y_max) / (2 * img_h)
    width = (x_max - x_min) / img_w
    height = (y_max - y_min) / img_h
    return 0, x_center, y_center, width, height  # Class ID is 0 for runway

In [76]:
# Process images and generate labels for train/val sets
for subset, image_list in [('train', train_images), ('val', val_images)]:
    for img_file in image_list:
        img_path = os.path.join(image_folder, img_file)

        if not os.path.exists(img_path):
            print(f"Image not found: {img_path}")
            continue

        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to load image: {img_file}")
            continue

        orig_h, orig_w = img.shape[:2]
        img_resized = cv2.resize(img, (640, 640))
        output_img_path = f'{output_folder}/images/{subset}/{img_file}'
        cv2.imwrite(output_img_path, img_resized)

        base_filename, _ = os.path.splitext(img_file)
        label_file = os.path.join(output_folder, 'labels', subset, base_filename + '.txt')

        try:
            row = df[df['image'].str.strip().str.lower() == img_file.strip().lower()].iloc[0]
            corners = row.iloc[-8:].values.astype(float)
            scaled_corners = scale_bbox(corners, (orig_w, orig_h), (640, 640))
            label_data = convert_to_yolo((640, 640), scaled_corners)

            with open(label_file, 'w') as f:
                f.write(" ".join(map(str, label_data)) + '\n')

        except IndexError:
            print(f"No matching metadata found for image: {img_file}")

In [77]:
import matplotlib.pyplot as plt

def visualize_resized_images_with_labels(image_path, corners):
    """Function to visualize resized images with their bounding boxes."""
    # Load and resize the image
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, (640, 640))

    # Draw the bounding box on the image
    points = [(int(corners[i]), int(corners[i + 1])) for i in range(0, 8, 2)]
    for j in range(len(points)):
        cv2.line(img_resized, points[j], points[(j + 1) % 4], (0, 255, 0), 2)

    # Display the image with the bounding box
    plt.figure(figsize=(6, 6))
    plt.imshow(cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

for subset in ['train', 'val']:
    print(f"Visualizing bounding boxes for {subset} set:")
    image_list = os.listdir(f'{output_folder}/images/{subset}')

    # Visualize up to 3 random images from the subset
    for img_file in random.sample(image_list, min(3, len(image_list))):
        img_path = os.path.join(output_folder, 'images', subset, img_file)

        # Load the corresponding label file
        label_file = os.path.join(
            output_folder, 'labels', subset, 
            img_file.replace('.jpeg', '.txt').replace('.jpg', '.txt')
        )

        with open(label_file, 'r') as f:
            label_data = f.readline().strip().split()

        # Print the label data to debug
        print(f"Label data for {img_file}: {label_data}")

        # Ensure the label is valid
        if len(label_data) == 5:
            # Extract class_id as integer and bounding box values as floats
            class_id = int(label_data[0])
            x_center, y_center, width, height = map(float, label_data[1:])

            # Convert YOLO format back to corner points for visualization
            img_w, img_h = 640, 640  # Resized dimensions
            x_min = int((x_center - width / 2) * img_w)
            x_max = int((x_center + width / 2) * img_w)
            y_min = int((y_center - height / 2) * img_h)
            y_max = int((y_center + height / 2) * img_h)

            # Prepare corner coordinates for visualization
            corners = [x_min, y_min, x_max, y_min, x_max, y_max, x_min, y_max]

            # Visualize the image with bounding box using the helper function
            visualize_resized_images_with_labels(img_path, corners)
        else:
            print(f"Skipping {img_file} due to invalid label format.")



Visualizing bounding boxes for train set:
Label data for BIRK_13_500_009.jpeg: ['0', '0.4852941176470589', '0.4592145015105741', '0.017973856209150353', '0.011329305135951716']


<Figure size 600x600 with 1 Axes>

Label data for BIRK_01_500_326.jpeg: ['0', '0.588031045751634', '0.3972809667673716', '0.05024509803921564', '0.03625377643504528']


<Figure size 600x600 with 1 Axes>

Label data for BIRK_13_500_158.jpeg: ['0', '0.36131535947712423', '0.4679003021148036', '0.01674836601307188', '0.014350453172205402']


...

<Figure size 600x600 with 1 Axes>

Label data for LFST_05_500_018.jpeg: ['0', '0.5516748366013072', '0.41597432024169195', '0.02083333333333339', '0.0154833836858006']


<Figure size 600x600 with 1 Axes>

Label data for BIRK_01_500_188.jpeg: ['0', '0.15972222222222224', '0.5521148036253777', '0.025326797385620915', '0.02794561933534743']


<Figure size 600x600 with 1 Axes>

In [78]:
#SKIP FOR NOW WILL HaRDCOODE YAM FILE
# Create YAML configuration file
#yaml_content = f"""path: {output_folder}
#train: images/train
#val: images/val
#nc: 1
#names: ['runway']
#"""

In [79]:
#SKIP FOR NOW WILL HaRDCOODE YAM FILE
#with open(f'{output_folder}/runway.yaml', 'w') as f:
   # f.write(yaml_content)

#print("Data preparation complete!")

In [80]:
# Initialize the YOLOv8 model and train it
model = YOLO('yolov8n.pt')
model.train(data=os.path.join(output_folder, 'runway.yaml'), epochs=10, imgsz=640)

# Test the model on a resized test image
test_image = r'C:\Users\Annabelle\Desktop\runway-bounding-box-detection NEW\test_image.jpeg'
img = cv2.imread(test_image)
img_resized = cv2.resize(img, (640, 640))
cv2.imwrite('test_image_resized.jpeg', img_resized)

# Run prediction on the resized test image
results = model.predict(source='test_image_resized.jpeg', show=True)
for r in results:
    print(f"Detected boxes: {r.boxes}")
    r.show()

Ultralytics 8.3.23  Python-3.11.9 torch-2.5.0+cpu CPU (AMD Ryzen 7 5800X 8-Core Processor)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset_demo\runway.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train18, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

train: Scanning C:\Users\Annabelle\Desktop\runway-bounding-box-detection NEW\dataset_demo\labels\train... 10 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10/10 [00:00<00:00, 3329.87it/s]

train: New cache created: C:\Users\Annabelle\Desktop\runway-bounding-box-detection NEW\dataset_demo\labels\train.cache



val: Scanning C:\Users\Annabelle\Desktop\runway-bounding-box-detection NEW\dataset_demo\labels\val... 10 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10/10 [00:00<00:00, 3330.93it/s]

val: New cache created: C:\Users\Annabelle\Desktop\runway-bounding-box-detection NEW\dataset_demo\labels\val.cache
Plotting labels to runs\detect\train18\labels.jpg... 


...

      10/10         0G      1.934      5.683      1.282         10        640: 100%|██████████| 1/1 [00:01<00:00,  1.78s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.57it/s]

                   all         10         10      0.002        0.6    0.00187   0.000832



10 epochs completed in 0.008 hours.
Optimizer stripped from runs\detect\train18\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train18\weights\best.pt, 6.2MB

Validating runs\detect\train18\weights\best.pt...
Ultralytics 8.3.23  Python-3.11.9 torch-2.5.0+cpu CPU (AMD Ryzen 7 5800X 8-Core Processor)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.59it/s]


                   all         10         10      0.002        0.6    0.00185   0.000849
Speed: 1.5ms preprocess, 49.9ms inference, 0.0ms loss, 5.8ms postprocess per image
Results saved to runs\detect\train18

image 1/1 c:\Users\Annabelle\Desktop\runway-bounding-box-detection NEW\test_image_resized.jpeg: 640x640 (no detections), 37.5ms
Speed: 2.0ms preprocess, 37.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Detected boxes: ultralytics.engine.results.Boxes object with attributes:

cls: tensor([])
conf: tensor([])
data: tensor([], size=(0, 6))
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([0, 6])
xywh: tensor([], size=(0, 4))
xywhn: tensor([], size=(0, 4))
xyxy: tensor([], size=(0, 4))
xyxyn: tensor([], size=(0, 4))
